In [ ]:
!pip install ktrain

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git #downloading dataset

In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ktrain
from ktrain import text

In [8]:
train_data_path = '/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx'
test_data_path = '/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx'

train = pd.read_excel(train_data_path, dtype=str)
test = pd.read_excel(test_data_path, dtype=str)

In [16]:
print(train.shape, test.shape)

(25000, 2) (25000, 2)


In [18]:
#Preprocessing our data for BERT which includes generating contextualised word embeddings (ELMo)
(X_train, y_train), (X_test, y_test), preprocess_info = text.texts_from_df(train_df=train,
                                                                            text_column='Reviews',
                                                                            label_columns='Sentiment',
                                                                            val_df=test,
                                                                            maxlen=400,
                                                                            preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [19]:
type(X_train), type(X_test), type(y_train), type(y_test) #len(X_train)=2

(list, list, numpy.ndarray, numpy.ndarray)

In [33]:
model = text.text_classifier(name='bert',
                             train_data=(X_train, y_train),
                             preproc=preprocess_info)

Is Multi-Label? False
maxlen is 400
done.


In [34]:
#Getting learning rate
learner = ktrain.get_learner(model=model,
                             train_data=(X_train, y_train),
                             val_data=(X_test, y_test),
                             batch_size=6)

In [ ]:
#Takes a lot of time
learner.lr_find()
learner.lr_plot()

In [35]:
checkpoint = '/content/IMDB-Movie-Reviews-Large-Dataset-50k/checkpoints/'

#Training
learner.fit_onecycle(lr=2e-5, epochs=2, checkpoint_folder=checkpoint)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - ETA: 0s - loss: 0.2665 - accuracy: 0.8908

KeyboardInterrupt: ignored

In [37]:
predictor = ktrain.get_predictor(learner.model, preprocess_info)

In [38]:
test_data = ['I kind of liked the movie, the acting was quite good!',
             'i can compare this movie to garbage, even a kids story would be better than this, so horrible',
             'loved the movie so much']

In [39]:
predictor.predict(test_data)

['pos', 'neg', 'pos']

In [40]:
#Saving model
predictor.save('/content/bert_model')